In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/
!git clone https://github.com/SoonbeomChoi/miniSynth.git
%cd miniSynth

In [ ]:
!pip install pretty_midi

In [1]:
import torch
import torch.nn as nn
from os import mkdir, path
from tqdm import tqdm

import config, data, preprocess
from model import Model

In [2]:
# Run preprocess
preprocess.main()

100%|██████████| 1/1 [00:00<00:00, 20560.31it/s]


In [3]:
if not path.exists(config.exp_path):
    mkdir(config.exp_path)

dataloader = data.load()
model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), config.learning_rate)
criterion = nn.L1Loss()

for epoch in range(int(config.stop_step/config.save_step)):
    model.train()
    progress_bar = tqdm(range(config.save_step), leave=False)
    for _ in progress_bar:
        note, mel = next(dataloader['train'])
        note = note.cuda()
        mel = mel.cuda()
        
        optimizer.zero_grad()
        mel_gen = model(note)
        loss = criterion(mel_gen, mel)
        loss.backward()
        optimizer.step()
        progress_bar.set_description(f"Loss - {loss.item():.4f}")

    torch.save(model.state_dict(), path.join(config.exp_path, 'checkpoint.pt'))

StopIteration: 